In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!pip install fastparquet
!sudo apt-get install libsnappy-dev
!python3 -m pip install  python-snappy
!python3 -m pip install pyarrow
!pip install tensorflow-addons
!pip install --upgrade numpy

     |████████████████████████████████| 1.2 MB 13.4 MB/s 
     |████████████████████████████████| 59 kB 6.3 MB/s 
     |████████████████████████████████| 125 kB 60.5 MB/s 
     |████████████████████████████████| 1.5 MB 67.9 MB/s 
  Created wheel for thrift: filename=thrift-0.15.0-cp37-cp37m-linux_x86_64.whl size=348186 sha256=281ec83d1507ec462ff62107635457f2154173a0f2f2bb4c3c21260566e15d35
  Stored in directory: /root/.cache/pip/wheels/ba/1f/8e/e6fd36837eecf3d1f2b23f1729477e8e06558d8d60b7093f51
Successfully built thrift
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  libsnappy-dev
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 27.2 kB of archives.
After this operation, 108 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsnappy-dev amd64 1.1.7-1 [27.2 kB]
Fetched 27.2 kB in 0s (546 kB/s)
debconf: unable to init

In [4]:
import os
os.chdir('/content/gdrive/My Drive/phyto_curves_reco')

import re
import pandas as pd
from pred_functions import predict
from keras.models import load_model
from time import time

from tensorflow_addons.optimizers import RectifiedAdam, Lookahead


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
# Model and nomenclature loading
cnn = load_model('/content/gdrive/My Drive/Models/SSLAMM/cnn_small')

In [6]:
os.chdir('/content/gdrive/My Drive/data/SSLAMM/')

In [25]:
tn = pd.read_csv('L2/Pulse/train_test_nomenclature.csv')
tn.columns = ['name', 'id']

# Define where to look the data at and where to store preds
export_folder = "full_data/"
export_files = os.listdir(export_folder)

pulse_regex = "_Pulse" 
files_to_pred = [file for file in export_files if re.search(pulse_regex, file) and re.search('.parq', file)] # The files containing the data to predict

In [26]:
files_to_pred.sort()
files_to_pred = files_to_pred[1570:1575]

In [27]:
# Create a log file in the destination folder: list of the already predicted files
preds_store_folder = "/content/gdrive/My Drive/Results/SSLAMM/Preds1"  # Where to store the predictions
log_path = preds_store_folder + "/pred_logs.txt" # Register where write the already predicted files

if not(os.path.isfile(log_path)):
    open(log_path, 'w+').close()

In [23]:
start_time = time()
nb_files_to_pred = len(files_to_pred)
print(nb_files_to_pred)

20


In [28]:
for idx, file in enumerate(files_to_pred):
    print('Currently predicting ' + file + ' ' + str(idx))
    path = export_folder + '/' + file
    is_already_pred = False
    
    # Check if file has already been predicted.
    with open(log_path, "r") as log_file:
        if file in log_file.read(): 
            is_already_pred = True
    
    if not(is_already_pred): # If not, perform the prediction
        # Predict the values
        predict(path, preds_store_folder,  cnn, tn,\
            is_ground_truth = False)

        # Write in the logs that this file is already predicted
        with open(log_path, "a") as log_file:
            log_file.write(file + '\n')

        step_time = time()
        average_pred_time = (step_time - start_time) / (idx + 1)
        remaining_time = average_pred_time * (nb_files_to_pred - idx - 1)
        print('Average per file pred time', average_pred_time, idx, 'files already predicted')
        print('Remaining time before end of pred', remaining_time)
          
    else:
        print(file, 'already predicted')

Currently predicting Labelled_Pulse6_2019-12-05 17h59.parq 0
Average per file pred time 1941.866759777069 0 files already predicted
Remaining time before end of pred 36895.46843576431
Currently predicting Labelled_Pulse6_2019-12-05 19h59.parq 1
Average per file pred time 1064.982437968254 1 files already predicted
Remaining time before end of pred 19169.683883428574
Currently predicting Labelled_Pulse6_2019-12-05 21h59.parq 2
Average per file pred time 768.493003209432 2 files already predicted
Remaining time before end of pred 13064.381054560345
Currently predicting Labelled_Pulse6_2019-12-05 23h59.parq 3
Average per file pred time 623.5694739222527 3 files already predicted
Remaining time before end of pred 9977.111582756042
Currently predicting Labelled_Pulse6_2019-12-06 01h59.parq 4
Average per file pred time 535.1707469463348 4 files already predicted
Remaining time before end of pred 8027.561204195023
